# <font face="Verdana" size=6 color='#6495ED'> ANÁLISE ESTATÍSTICA DE DADOS

<font face="Verdana" size=3 color='#40E0D0'> Profa. Larissa Driemeier

<center><img src='https://drive.google.com/uc?export=view&id=1nW_7p_LyFhbR0ipjSekPcAj6kDoyK73R' width="800"></center>

Este notebook faz parte da aula 04 do curso [IAD-001](https://alunoweb.net/moodle/course/view.php?id=3731#section-2).



# Importando bibliotecas

In [ ]:
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
import numpy as np
from sklearn.decomposition import TruncatedSVD
import math

In [ ]:
color_g1 = 'darkolivegreen'
color_g2 = 'olive'
color_g3 = 'forestgreen'
color_g4 = 'green'
color_p1 = 'darkmagenta'
color_p2 = 'mediumorchid'
color_p3 = 'thistle'
color_p4 = 'lavandablush'
color_b1 = 'navy'
color_b2 = 'royalblue'
color_b4 = 'slateblue'
color_r1 = 'indianred'

#Autovalores e autovetores

A definição de autovalores e autovetores será bastante usada nessa aula: para uma matriz quadrada $\mathbf A_{n\times n}$, um escalar $\lambda$ é chamado de autovalor de se houver um vetor não nulo $\mathbf x$ tal que
$$
\mathbf{Ax}=\lambda\mathbf{x}.
$$

Tal vetor, $\mathbf x$, é chamado de autovetor de $\mathbf A$ correspondente ao autovalor $\lambda$. Às vezes, nos referimos ao par como um par de autovalores. Por exemplo,
$$
\mathbf{Ax}=\begin{bmatrix}  3 & 1 \\1&3\end{bmatrix} \begin{bmatrix} 1\\1 \end{bmatrix} =\begin{bmatrix} 4\\4 \end{bmatrix}=4\begin{bmatrix} 1\\1 \end{bmatrix} = \lambda\mathbf{x}
$$

Daqui segue que $\mathbf x$ é um autovetor de $\mathbf A$ e o autovalor correspondente é $\lambda$.


 Porém, desta vez, iremos utilizar a biblioteca `np.linalg` para calcular autovalores e autovetores. Caso essa pequena recordação não seja suficiente para você, volte à aula anterior e relembre.

<center><img src='https://drive.google.com/uc?export=view&id=16mcBy-SRAuzQvcxg7xPr_v9xqyKBVMzS' width="400"></center>


Para a biblioteca `linalg` retornar autovalores `lam` e autovetores `u` da matriz `A`, tem-se o comando:
```
lam,u = np.linalg.eig(A)
```

Na resposta abaixo tem-se o primeiro invariante e seu respectivo autovetor em vermelho e o segundo em azul.

<center><img src='https://drive.google.com/uc?export=view&id=1qSHnuXHDRnlABNBUSW6xrFTjtbWlGtgq' width="200"></center>



In [ ]:
A = np.array([[3,2],[0,2]])
# Cálculo de autovalores e autovetores
lam, u = np.linalg.eig(A)
print("autovalores=\n", np.round(lam, 4))
print("autovetores=\n", np.round(u, 4))

In [ ]:
A = np.array([[7,3],[3,-1]])
# Cálculo de autovalores e autovetores
lam, u = np.linalg.eig(A)
print("autovalores=\n", np.round(lam, 4))
print("autovetores=\n", np.round(u, 4))

In [ ]:
A = np.array([[3, 1],
              [1, 0.8]])
# Cálculo de autovalores e autovetores
lam, u = np.linalg.eig(A)
print("autovalores=\n", np.round(lam, 4))
print("autovetores=\n", np.round(u, 4))

#Transformação

A matriz de transformação $\boldsymbol{A}$ *transformou* o círculo da figura abaixo em uma elipse.

![Transf_Circle_Asym](https://drive.google.com/uc?export=view&id=1Vi_LgXQ9I-0UcTk-f8BShmR6wPIPoyEQ)




Para verificar a diferença entre uma matriz simétrica e não simétrica, substitua a matriz  $\boldsymbol{A}$ do código abaixo pelas matrizes:

\begin{equation}
\boldsymbol{A} = \begin{bmatrix}3 & 2 \\ 0 & 2 \end{bmatrix}
\end{equation}
e depois,
\begin{equation}
\boldsymbol{A} = \begin{bmatrix}3 & 1 \\ 1 & 2 \end{bmatrix}
\end{equation}

In [ ]:
# Cálculo de autovalores e autovetores
A = np.array([[3, 2],
             [0, 2]])
#A = np.array([[3, 1],
#              [1, 2]])
lam, u = np.linalg.eig(A)
print("autovalores=\n", np.round(lam, 4))
print("autovetores=\n", np.round(u, 4))


É possível perceber que, quando a matriz é simétrica, os autovetores estão alinhados com os eixos principal e secundário da elipse.

![Wall-E](https://drive.google.com/uc?export=view&id=1SikuSQ8_p4O4Wux56Ih00GrQtPCHy5nr)

Para você entender melhor o que definimos acima, o algoritmo abaixo foi adaptado do [link](https://towardsdatascience.com/understanding-singular-value-decomposition-and-its-application-in-data-science-388a54be95d) e é usado para gerar gráficos e tornar as trasnformações mais visuais.

__Não se preocupe muito em entender o algoritmo. Entenda somente as imagens que foram geradas.__

In [ ]:
def graph(u,A):
  # Creating the vectors for a circle and storing them in x
  xi1 = np.linspace(-1.0, 1.0, 100)
  xi2 = np.linspace(1.0, -1.0, 100)
  yi1 = np.sqrt(1 - xi1**2)
  yi2 = -np.sqrt(1 - xi2**2)

  xi = np.concatenate((xi1, xi2),axis=0)
  yi = np.concatenate((yi1, yi2),axis=0)
  x = np.vstack((xi, yi))

  # getting a sample vector from x
  x_sample1 = u[:,0]
  x_sample2 = u[:,1]

  t = A @ x  # Vectors in t are the transformed vectors of x

  origin = [0], [0] # origin point

  t_sample1 = A @ x_sample1
  t_sample2 = A @ x_sample2

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,15))

  plt.subplots_adjust(wspace=0.4)

  # Plotting x
  ax1.plot(x[0,:], x[1,:], color=color_b2)
  ax1.quiver(*origin, x_sample1[0], x_sample1[1], color=[color_g1], width=0.012, angles='xy', scale_units='xy', scale=1)
  ax1.quiver(*origin, x_sample2[0], x_sample2[1], color=[color_g2], width=0.012, angles='xy', scale_units='xy', scale=1)
  ax1.set_xlabel('x', fontsize=14)
  ax1.set_ylabel('y', fontsize=14)
  ax1.set_xlim([-4,4])
  ax1.set_ylim([-4,4])
  ax1.set_aspect('equal')
  ax1.grid(True)
  ax1.set_axisbelow(True)
  ax1.set_title("Vetores originais")
  ax1.axhline(y=0, color='k')
  ax1.axvline(x=0, color='k')
  ax1.text(x_sample1[0], x_sample1[1], "$\mathbf{u_1}$", color=color_g1, fontsize=14)
  ax1.text(x_sample2[0], x_sample2[1], "$\mathbf{u_2}$", color=color_g2, fontsize=14)

  # Plotting t
  ax2.plot(t[0, :], t[1, :], color=color_b2)
  ax2.quiver(*origin, t_sample1[0], t_sample1[1], color=[color_g1], width=0.012, angles='xy', scale_units='xy', scale=1)
  ax2.quiver(*origin, t_sample2[0], t_sample2[1], color=[color_g2], width=0.012, angles='xy', scale_units='xy', scale=1)
  max_value = np.max(np.array([math.ceil(t_sample2[0]),math.ceil(t_sample1[0]),math.ceil(t_sample2[1]),math.ceil(t_sample1[1])]))
  ax2.set_xlim([-max_value,max_value])
  ax2.set_ylim([-max_value,max_value])

  ax2.set_aspect('equal')
  ax2.grid(True)
  ax2.set_axisbelow(True)
  ax2.set_title("Novos vetores após transformação")
  ax2.axhline(y=0, color='k')
  ax2.axvline(x=0, color='k')
  ax2.text(t_sample1[0], t_sample1[1], "$\mathbf{t_1}$", color=color_g1, fontsize=14)
  ax2.text(t_sample2[0], t_sample2[1], "$\mathbf{t_2}$", color=color_g2, fontsize=14)
  plt.savefig('2.png', dpi=300, bbox_inches='tight')

  plt.show()

In [ ]:
graph(u,A)

#Decomposição em autovalores
Toda matriz simétrica quadrada $\boldsymbol{A}_{n \times n}$ pode ser decomposta em,
$$
\boldsymbol{A}=\boldsymbol{U}\boldsymbol{S}\boldsymbol{U}^T
$$
ou, equivalentemente,
\begin{equation}
\boldsymbol{A} = \begin{bmatrix} a_{1,1} & a_{1,2} & \ldots & a_{1,n} \\  
a_{2,1} & a_{2,2} & \ldots & ax_{2,n} \\  
 \vdots & \vdots & \ddots & \vdots & \vdots\\  
a_{n,1} & a_{n,2} & \ldots & a_{n,n} \end{bmatrix}=
\begin{bmatrix} \vdots & \vdots & \ldots & \vdots \\  
\boldsymbol{u}_1 & \boldsymbol{u}_2 & \ldots & \boldsymbol{u}_n \\  
 \vdots & \vdots & \ddots & \vdots
 \end{bmatrix}
 \begin{bmatrix} \lambda_1 & 0 & \ldots & 0 \\  
0 & \lambda_2 & \ldots & 0\\  
 \vdots & \vdots & \ddots & \vdots
 \\  
0 & \vdots & 0 &  \lambda_n
 \end{bmatrix}
 \begin{bmatrix} \ldots & \boldsymbol{u}_1^T & \ldots  \\  
\ldots & \boldsymbol{u}_2^T & \ldots \\  
 \vdots & \vdots & \ddots &  \\  
\ldots & \boldsymbol{u}_n^T & \ldots
 \end{bmatrix}
\end{equation}
onde $\boldsymbol{u}_i$ são $n$ LI autovetores de $\boldsymbol{A}$ e $\lambda_i$ são os autovalores de $\boldsymbol{A}$.

Pode-se mostrar que a matriz simétrica $\boldsymbol{A}$ pode ser escrita como,
$$
𝑨=\lambda_1 \boldsymbol{u}_1 \boldsymbol{u}_1^T+\lambda_2 \boldsymbol{u}_2 \boldsymbol{u}_2^T+\cdots +\lambda_𝑘 \boldsymbol{u}_𝑘 \boldsymbol{u}_k^T+\cdots+\lambda_𝑛 \boldsymbol{u}_n \boldsymbol{u}_n^T
$$
onde, para o caso de matriz simétrica definida positiva (todos autovalores são positivos),
$$
\lambda_1 \geq \lambda_2 \geq \cdots \geq \lambda_k \cdots \geq \lambda_n \geq 0
$$
Quanto maior o autovalor, maior o comprimento do vetor resultante $\lambda_𝑘 \boldsymbol{u}_𝑘 \boldsymbol{u}_k^T$ e mais peso é dado à sua matriz correspondente $\boldsymbol{u}_𝑖 \boldsymbol{u}_𝑖^𝑇$. Assim, podemos aproximar nossa matriz simétrica original $\boldsymbol{A}$ somando os termos que têm os maiores autovalores,
$$
\boldsymbol{A}\approx \boldsymbol{A}_k = \lambda_1 \boldsymbol{u}_1 \boldsymbol{u}_1^T+\lambda_2 \boldsymbol{u}_2 \boldsymbol{u}_2^T+\cdots +\lambda_𝑘 \boldsymbol{u}_𝑘 \boldsymbol{u}_k^T
$$


## Interpretação importante

Agora, iremos calcular $\boldsymbol{A}_1 = \lambda_1 \boldsymbol{u}_1 \boldsymbol{u}_1^T$ para a matriz

\begin{equation}
\boldsymbol{A} = \begin{bmatrix}3 & 1 \\ 1 & 2 \end{bmatrix}
\end{equation}
e aplicaremos em nosso círculo, para entendermos o papel de cada termo da matriz $\boldsymbol{A}$ na transformação.

In [ ]:
A = np.array([[3, 1],
              [1, 2]])
lam, u = np.linalg.eig(A)
print("autovalores=\n", np.round(lam, 4))
print("autovetores=\n", np.round(u, 4))
S=np.array([[lam[0], 0],
              [0, lam[1]]])
u1 = np.array(u[:,0])
u2 =np.array(u[:,1])
P=np.array([u1,u2]).T


t = P @ S @ P.T
print("Recupera-se a matriz A\n",t)

In [ ]:
A1 = lam[0]*np.outer(u1,u1)
print('A1\n',A1)
A2 = lam[1]*np.outer(u2,u2)
print('A2\n',A2)

Como é possível observar, na figura abaixo, o círculo inicial é alongado ao longo de $\boldsymbol{u}_1$ e reduzido a zero ao longo de $\boldsymbol{u}_2$. Portanto, o resultado dessa transformação é uma linha reta, não uma elipse.

Isso é consistente com o fato de que $\boldsymbol{A}_1$ é uma matriz de projeção em $\boldsymbol{u}_1$, então o resultado deve ser uma linha reta ao longo de $\boldsymbol{u}_1$.



In [ ]:
graph(u,A)

Veja o exemplo 3D:

\begin{equation}
\boldsymbol{A} = \begin{bmatrix}4 & 1 & 3 \\ 1 & 7 & -2 \\ 3 & -2 & 4 \end{bmatrix}
\end{equation}
e aplicaremos em nosso círculo, para entendermos o papel de cada termo da matriz $\boldsymbol{A}$ na transformação.

In [ ]:
A = np.array([[4, 1, 3],
              [8, 7, -2],
              [1, 5, 4]])

In [ ]:
# extraído do link: https://towardsdatascience.com/understanding-singular-value-decomposition-and-its-application-in-data-science-388a54be95d
# Desenha imagens 3D

def graph3D(A, i=70, j=30):
  U, s, VT = np.linalg.svd(A)
  Sigma = np.zeros((A.shape[0], A.shape[1]))

  Sigma[:min(A.shape[0],A.shape[1]), :min(A.shape[0],A.shape[1])] = np.diag(s)
  A_rank1 = U[:, :1] @ Sigma[:1, :1] @ VT[:1, :]
  A_rank2 = U[:, :2] @ Sigma[:2, :2] @ VT[:2, :]

  u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:20j]
  xi = np.cos(u)*np.sin(v)
  yi = np.sin(u)*np.sin(v)
  zi = np.cos(v)

  x = np.vstack((xi.reshape(1,-1), yi.reshape(1,-1), zi.reshape(1,-1)))

  t1 = A @ x
  t2 = A_rank2 @ x
  t3 = A_rank1 @ x

  tx1 = t1[0,:].reshape(30,20)
  ty1 = t1[1,:].reshape(30,20)
  tz1 = t1[2,:].reshape(30,20)

  tx2 = t2[0,:].reshape(30,20)
  ty2 = t2[1,:].reshape(30,20)
  tz2 = t2[2,:].reshape(30,20)

  tx3 = t3[0,:].reshape(30,20)
  ty3 = t3[1,:].reshape(30,20)
  tz3 = t3[2,:].reshape(30,20)

  fig = plt.figure(figsize=(20,10))
  plt.subplots_adjust(wspace=0.03)
  ax1 = fig.add_subplot(131, projection='3d')
  ax2 = fig.add_subplot(132, projection='3d')
  ax3 = fig.add_subplot(133, projection='3d')

  # plot 1
  ax1.plot_wireframe(tx1, ty1, tz1, color=color_b2, alpha=0.2)
  ax1.quiver(0, 0, 0, s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax1.text(s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], "$\sigma_1\mathbf{u_1}$", fontsize=18, color=color_g1)
  ax1.quiver(0, 0, 0, s[1]*U[0,1], s[1]* U[1,1], s[1]*U[2,1], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax1.text(s[1]*U[0,1], s[1]* U[1,1], s[1]*U[2,1], "$\sigma_2\mathbf{u_2}$", fontsize=18, color=color_g1)
  ax1.quiver(0, 0, 0, s[2]*U[0,2], s[2]* U[1,2], s[2]*U[2,2], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax1.text(s[2]*U[0,2], s[2]* U[1,2], s[2]*U[2,2], "$\sigma_3\mathbf{u_3}$", fontsize=18, color=color_g1)

  ax1.set_xlabel("x", fontsize=18)
  ax1.set_ylabel("y", fontsize=18)
  ax1.set_zlabel("z", fontsize=18)
  ax1.set_xlim([-10,5])
  ax1.set_ylim([-12, 12])
  ax1.set_zlim([-6, 6])
  ax1.set_title(r"$\mathbf{Ax}$", fontsize=18)

  # plot 2
  ax2.plot_surface(tx2, ty2, tz2, color=color_b2, alpha=0.2)
  ax2.quiver(0, 0, 0, s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax2.text(s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], "$\sigma_1\mathbf{u_1}$", fontsize=18, color=color_g1)
  ax2.quiver(0, 0, 0, s[1]*U[0,1], s[1]* U[1,1], s[1]*U[2,1], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax2.text(s[1]*U[0,1], s[1]* U[1,1], s[1]*U[2,1], "$\sigma_2\mathbf{u_2}$", fontsize=18, color=color_g1)
  ax2.set_xlim([-10, 5])
  ax2.set_ylim([-12, 12])
  ax2.set_zlim([-6, 6])
  ax2.set_xlabel("x", fontsize=18)
  ax2.set_ylabel("y", fontsize=18)
  ax2.set_zlabel("z", fontsize=18)
  ax2.set_title(r"$\mathbf{A}_{2}\mathbf{x}=\sigma_1\mathbf{u}_1\mathbf{u}_1^T\mathbf{x}+\sigma_2\mathbf{u}_2\mathbf{u}_2^T\mathbf{x}$", fontsize=18)

  # plot 3
  ax3.plot_wireframe(tx3, ty3, tz3, color=color_b2, alpha=0.02)
  ax3.quiver(0, 0, 0, s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax3.text(s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], "$\sigma_1\mathbf{u_1}$", fontsize=18, color=color_g1)
  ax3.set_xlim([-10, 5])
  ax3.set_ylim([-12, 12])
  ax3.set_zlim([-6, 6])
  ax3.set_xlabel("x", fontsize=18)
  ax3.set_ylabel("y", fontsize=18)
  ax3.set_zlabel("z", fontsize=18)
  ax3.set_title(r"$\mathbf{A}_{1}\mathbf{x}=\sigma_1\mathbf{u}_1\mathbf{u}_1^T\mathbf{x}$", fontsize=18)

  ax1.view_init(i, j)
  ax2.view_init(i, j)
  ax3.view_init(i, j)

  plt.draw()

In [ ]:
i,j = 50, 120
graph3D(A,i,j)

# Valores singulares de uma matriz

Chamamos de valores singulares da matriz $\boldsymbol{A}_{m \times n}$, $\lambda_1, \lambda_2, \cdots, \lambda_r$, as $r$ raízes quadradas dos autovalores não nulos $\boldsymbol{A}\boldsymbol{A}^T$ ou $\boldsymbol{A}^T\boldsymbol{A}$;
$$
\sigma_i=\sqrt{\lambda_i}
$$
Os vetores singulares $\boldsymbol{u}_i$ à esquerda da matriz $\boldsymbol{A}$ são os autovetores de $\boldsymbol{A}\boldsymbol{A}^T$, $m \times m$.
Os vetores singulares $\boldsymbol{v}_i$ à direita da matriz $\boldsymbol{A}$ são os autovetores de $\boldsymbol{A}^T\boldsymbol{A}$, $n \times n$.  

Os autovetores de  $\boldsymbol{A}\boldsymbol{A}^T$ formam as colunas da matriz $\boldsymbol{U}$ e os autovetores de $\boldsymbol{A}^T\boldsymbol{A}$ formam as colunas da matriz $\boldsymbol{V}$. Como são simétricas, os autovetores são perpendiculares entre si e, se possuem comprimento unitário, tem-se que:
 $$\boldsymbol{U}^T\boldsymbol{U}=\boldsymbol{I}$$
  $$\boldsymbol{V}^T\boldsymbol{V}=\boldsymbol{I}$$


##  Um pouco mais sobre a matriz $ A^T A $ (ou $ A A^T $)
Seja $ A $ uma matriz $ m \times n $. A matriz $ A^T A $ é sempre simétrica porque:

$$
(A^T A)^T = A^T (A^T)^T = A^T A.
$$

Além disso, ela é **semidefinida positiva**, pois para qualquer vetor $ x $:

$$
x^T (A^T A) x = (Ax)^T (Ax) = \|Ax\|^2 \geq 0.
$$

Isso significa que seus autovalores são **reais e não negativos**.

Seja $ v_i $ um autovetor de $ A^T A $ com autovalor $ \lambda_i $,

$$
A^T A v_i = \lambda_i v_i.
$$

Veja que,

$$
\| A v_i \|^2 = v_i^T A^T A v_i = \lambda_i v_i^T v_i = \lambda_i \| v_i \|^2.
$$

Se $ \lambda_i > 0 $, isso implica que $ A v_i $ **não é nulo**, ou seja, o vetor $ v_i $ é mapeado por $ A $ para uma nova direção $ A v_i $, que é alongada proporcionalmente a $ \sqrt{\lambda_i} $. Pode-se mostrar que os $ A v_i $ representam direções principais dos alongamentos da transformação $Ax$, onde $\||x\||=1$.


In [ ]:
A = np.array([[3, 2],
              [0, 2]])
ATA = A.T @ A
AAT = A @ A.T

In [ ]:
# Cálculo de autovalores e autovetores
lamu, u = np.linalg.eig(AAT)
print("AA^T=\n", np.round(AAT, 4))
print("autovalores=\n", np.round(lamu, 4))
print("autovetores=\n", np.round(u, 4))

Lembrem-se que os autovetores são normalizados, de modo que a coluna `u[:,i]` é o autovetor correspondente ao autovalor `lam[i]`.

In [ ]:
# Cálculo de autovalores e autovetores
lamv, v = np.linalg.eig(ATA)
print("A^TA=\n", np.round(ATA, 4))
print("autovalores=\n", np.round(lamv, 4))
print("autovetores=\n", np.round(v, 4))

A orientação dos autovetores pode ocasionalmente diferir, mas tudo bem — afinal, um autovetor é invariante em escala.

In [ ]:
graph(u,AAT)

In [ ]:
graph(v,ATA)

Teste agora com a matriz,

\begin{equation}
\mathbf{A} = \begin{bmatrix} 3 & 2 & 2 \\
                             2 & 3 & -2 \end{bmatrix}
\end{equation}

In [ ]:
A = np.array([[3, 2, 2],
              [2, 3, -2]])
ATA = A.T @ A
AAT = A @ A.T
print("A^TA\n",ATA)
print("AA^T\n", AAT)

In [ ]:
# Cálculo de autovalores e autovetores
lamv, v = np.linalg.eig(ATA)
print("A^TA=\n", np.round(ATA, 4))
print("autovalores=\n", np.round(lamv, 4))
print("autovetores=\n", np.round(v, 4))

In [ ]:
# Cálculo de autovalores e autovetores
lamu, u = np.linalg.eig(AAT)
print("AA^T=\n", np.round(AAT, 4))
print("autovalores=\n", np.round(lamu, 4))
print("autovetores=\n", np.round(u, 4))

#Decomposição em valor singular

O SVD (Singular Value Decomposition) é um método para a fatorar matrizes:
$$
\boldsymbol{A}=\boldsymbol{U}\boldsymbol{S}\boldsymbol{V}^T
$$
onde $\boldsymbol{U}$ e $\boldsymbol{V}$ são as matrizes ortogonais que definimos e $\boldsymbol{S}$ é diagonal.

Os valores da matriz diagonal $\boldsymbol{S}$ são os valores singulares que acabamos de aprender, e por isso a decomposição recebe este nome.

O número de valores singulares diferentes de zero é igual ao rank da matriz $\boldsymbol{A}$.

## Exercício

Agora, resolva você:

\begin{equation}
\boldsymbol{A} = \begin{bmatrix}4 & 1 & 3 \\ 8 & 7 & -2 \\ 1 & 5 & 4 \end{bmatrix}
\end{equation}
e aplicaremos em nosso círculo, para entendermos o papel de cada termo da matriz $\boldsymbol{A}$ na transformação.

In [ ]:
A = np.array([[4, 1, 3],
              [8, 7, -2],
              [1, 5, 4]])

In [ ]:
# extraído do link: https://towardsdatascience.com/understanding-singular-value-decomposition-and-its-application-in-data-science-388a54be95d
# Desenha imagens 3D

def graph3D(A, i=70, j=30):
  U, s, VT = np.linalg.svd(A)
  Sigma = np.zeros((A.shape[0], A.shape[1]))

  Sigma[:min(A.shape[0],A.shape[1]), :min(A.shape[0],A.shape[1])] = np.diag(s)
  A_rank1 = U[:, :1] @ Sigma[:1, :1] @ VT[:1, :]
  A_rank2 = U[:, :2] @ Sigma[:2, :2] @ VT[:2, :]

  u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:20j]
  xi = np.cos(u)*np.sin(v)
  yi = np.sin(u)*np.sin(v)
  zi = np.cos(v)

  x = np.vstack((xi.reshape(1,-1), yi.reshape(1,-1), zi.reshape(1,-1)))

  t1 = A @ x
  t2 = A_rank2 @ x
  t3 = A_rank1 @ x

  tx1 = t1[0,:].reshape(30,20)
  ty1 = t1[1,:].reshape(30,20)
  tz1 = t1[2,:].reshape(30,20)

  tx2 = t2[0,:].reshape(30,20)
  ty2 = t2[1,:].reshape(30,20)
  tz2 = t2[2,:].reshape(30,20)

  tx3 = t3[0,:].reshape(30,20)
  ty3 = t3[1,:].reshape(30,20)
  tz3 = t3[2,:].reshape(30,20)

  fig = plt.figure(figsize=(20,10))
  plt.subplots_adjust(wspace=0.03)
  ax1 = fig.add_subplot(131, projection='3d')
  ax2 = fig.add_subplot(132, projection='3d')
  ax3 = fig.add_subplot(133, projection='3d')

  # plot 1
  ax1.plot_wireframe(tx1, ty1, tz1, color=color_b2, alpha=0.2)
  ax1.quiver(0, 0, 0, s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax1.text(s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], "$\sigma_1\mathbf{u_1}$", fontsize=18, color=color_g1)
  ax1.quiver(0, 0, 0, s[1]*U[0,1], s[1]* U[1,1], s[1]*U[2,1], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax1.text(s[1]*U[0,1], s[1]* U[1,1], s[1]*U[2,1], "$\sigma_2\mathbf{u_2}$", fontsize=18, color=color_g1)
  ax1.quiver(0, 0, 0, s[2]*U[0,2], s[2]* U[1,2], s[2]*U[2,2], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax1.text(s[2]*U[0,2], s[2]* U[1,2], s[2]*U[2,2], "$\sigma_3\mathbf{u_3}$", fontsize=18, color=color_g1)

  ax1.set_xlabel("x", fontsize=18)
  ax1.set_ylabel("y", fontsize=18)
  ax1.set_zlabel("z", fontsize=18)
  ax1.set_xlim([-10,5])
  ax1.set_ylim([-12, 12])
  ax1.set_zlim([-6, 6])
  ax1.set_title(r"$\mathbf{Ax}$", fontsize=18)

  # plot 2
  ax2.plot_surface(tx2, ty2, tz2, color=color_b2, alpha=0.2)
  ax2.quiver(0, 0, 0, s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax2.text(s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], "$\sigma_1\mathbf{u_1}$", fontsize=18, color=color_g1)
  ax2.quiver(0, 0, 0, s[1]*U[0,1], s[1]* U[1,1], s[1]*U[2,1], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax2.text(s[1]*U[0,1], s[1]* U[1,1], s[1]*U[2,1], "$\sigma_2\mathbf{u_2}$", fontsize=18, color=color_g1)
  ax2.set_xlim([-10, 5])
  ax2.set_ylim([-12, 12])
  ax2.set_zlim([-6, 6])
  ax2.set_xlabel("x", fontsize=18)
  ax2.set_ylabel("y", fontsize=18)
  ax2.set_zlabel("z", fontsize=18)
  ax2.set_title(r"$\mathbf{A}_{2}\mathbf{x}=\sigma_1\mathbf{u}_1\mathbf{v}_1^T\mathbf{x}+\sigma_2\mathbf{u}_2\mathbf{v}_2^T\mathbf{x}$", fontsize=18)

  # plot 3
  ax3.plot_wireframe(tx3, ty3, tz3, color=color_b2, alpha=0.02)
  ax3.quiver(0, 0, 0, s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], color=color_g1, length=1, arrow_length_ratio=0.07)
  ax3.text(s[0]*U[0,0], s[0]* U[1,0], s[0]*U[2,0], "$\sigma_1\mathbf{u_1}$", fontsize=18, color=color_g1)
  ax3.set_xlim([-10, 5])
  ax3.set_ylim([-12, 12])
  ax3.set_zlim([-6, 6])
  ax3.set_xlabel("x", fontsize=18)
  ax3.set_ylabel("y", fontsize=18)
  ax3.set_zlabel("z", fontsize=18)
  ax3.set_title(r"$\mathbf{A}_{1}\mathbf{x}=\sigma_1\mathbf{u}_1\mathbf{v}_1^T\mathbf{x}$", fontsize=18)

  ax1.view_init(i, j)
  ax2.view_init(i, j)
  ax3.view_init(i, j)

  plt.draw()

In [ ]:
i,j = 50, 120
graph3D(A,i,j)

In [ ]:
U, S, VT = np.linalg.svd(A)
print(S)
print(U)
print(VT.T)

# Lição de casa

<center><img src='https://drive.google.com/uc?export=view&id=1_1jsA41H4nNIgig_4HGoq0BPhksHkm0W' width="400"></center>

Antes de fazer a lição, estude a aula. Responda às questões manualmente:

1. Encontre os valores singulares das matrizes abaixo:

$$
A_1 =
\begin{bmatrix}
1 & 0 \\
0 & -3
\end{bmatrix}
$$

$$
A_2 =
\begin{bmatrix}
-5 & 0 \\
0 & 0
\end{bmatrix}
$$

---

2. Construa o SVD de cada matriz abaixo:

$$
A_3 =
\begin{bmatrix}
2 & -1 \\
2 & 2
\end{bmatrix}
$$

$$
A_4 =
\begin{bmatrix}
7 & 1 \\
0 & 0 \\
5 & 5
\end{bmatrix}
$$

$$
A_5 =
\begin{bmatrix}
1 & 0 & 1 & 0 \\
0 & 1 & 0 & 1
\end{bmatrix}
$$


---


3. (OPCIONAL) Com os dados de altura e peso de homens e mulheres, que você pode retirar do [link](https://www.kaggle.com/mustafaali96/weight-height/version/1),
 * Ache a matriz $\boldsymbol{A}$;
 * Ache os valores singulares;
 * Ache $\boldsymbol{U}$,$\boldsymbol{D}$ e $\boldsymbol{V}$;
 * Escreva a matriz na forma: $\boldsymbol{A}_k = \sigma_1\boldsymbol{u}_1\boldsymbol{v}_1^𝑇 + \cdots + \sigma_k\boldsymbol{u}_k\boldsymbol{v}_k^𝑇$;
 * Discuta suas conclusões, analisando a matriz $\boldsymbol{A}_k$.